In [37]:
print("hi")

hi


## Suppress Warnings

In [38]:
import warnings
warnings.filterwarnings('ignore')


## Import Libraries

In [39]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


## Set Image Size, Dataset Paths, Batch Size, Epochs

In [40]:
IMAGE_SIZE = (300, 300)
train_path = "Datasets/train"
test_path = "Datasets/test"
BATCH_SIZE = 16
EPOCHS = 10


## Create Training Data Generator

In [41]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=25,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)


## Create Testing Data Generator

In [42]:
test_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)


## Load Training Images

In [43]:
train_set = train_gen.flow_from_directory(
    train_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 5216 images belonging to 2 classes.


## Load Testing Images

In [44]:
test_set = test_gen.flow_from_directory(
    test_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 624 images belonging to 2 classes.


## Load EfficientNetB3 (Without Top Layer)

In [55]:
base = EfficientNetB3(
    weights="imagenet",
    include_top=False,
    input_shape=IMAGE_SIZE + (3,)
)


## Freeze Base Model

In [46]:
base.trainable = False

## Build Custom Classification Head

In [47]:
x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.35)(x)
output = Dense(2, activation="softmax")(x)

model = Model(inputs=base.input, outputs=output)


## Compile Model

In [48]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


## View Model Summary

In [49]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_4         │ (None, 300, 300,  │          0 │ input_layer_2[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_2     │ (None, 300, 300,  │          7 │ rescaling_4[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_5         │ (None, 300, 300,  │          0 │ normalization_2[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 301, 301,  │          0 │ rescaling_5[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 150, 150,  │      1,080 │ stem_conv_pad[0]… │
│                     │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 150, 150,  │        160 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 150, 150,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 150, 150,  │        360 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 150, 150,  │        160 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 150, 150,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 40)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 40)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 10)  │        410 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 40)  │        440 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 150, 150,  │          0 │ block1a_activati… │
│ (Multiply)          │ 40)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 150, 150,  │        960 │ block1a_se_excit

 Total params: 10,786,609 (41.15 MB)

 Trainable params: 3,074 (12.01 KB)

 Non-trainable params: 10,783,535 (41.14 MB)

## Setup Model Checkpoint

In [50]:
checkpoint = ModelCheckpoint(
    "pneumonia_optimized_Dibyendu.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)


## Setup EarlyStopping

In [51]:
early_stop = EarlyStopping(
    monitor="val_accuracy",
    patience=6,
    restore_best_weights=True,
    verbose=1
)


## Setup Learning Rate Scheduler

In [52]:
lr_reduce = ReduceLROnPlateau(
    monitor="val_loss",
    patience=3,
    factor=0.2,
    min_lr=1e-7,
    verbose=1
)


## Train Model

In [53]:
r = model.fit(
    train_set,
    validation_data=test_set,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stop, lr_reduce]
)


Epoch 1/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 722ms/step - accuracy: 0.8433 - loss: 0.3398
Epoch 1: val_accuracy improved from None to 0.89583, saving model to pneumonia_optimized_Dibyendu.h5


326/326 ━━━━━━━━━━━━━━━━━━━━ 274s 810ms/step - accuracy: 0.8988 - loss: 0.2399 - val_accuracy: 0.8958 - val_loss: 0.2782 - learning_rate: 0.0010
Epoch 2/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 749ms/step - accuracy: 0.9334 - loss: 0.1710
Epoch 2: val_accuracy did not improve from 0.89583
326/326 ━━━━━━━━━━━━━━━━━━━━ 270s 829ms/step - accuracy: 0.9377 - loss: 0.1596 - val_accuracy: 0.8894 - val_loss: 0.2580 - learning_rate: 0.0010
Epoch 3/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 769ms/step - accuracy: 0.9455 - loss: 0.1410
Epoch 3: val_accuracy improved from 0.89583 to 0.91827, saving model to pneumonia_optimized_Dibyendu.h5


326/326 ━━━━━━━━━━━━━━━━━━━━ 292s 897ms/step - accuracy: 0.9436 - loss: 0.1415 - val_accuracy: 0.9183 - val_loss: 0.2272 - learning_rate: 0.0010
Epoch 4/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 762ms/step - accuracy: 0.9472 - loss: 0.1348
Epoch 4: val_accuracy did not improve from 0.91827
326/326 ━━━━━━━━━━━━━━━━━━━━ 275s 845ms/step - accuracy: 0.9461 - loss: 0.1329 - val_accuracy: 0.8814 - val_loss: 0.2607 - learning_rate: 0.0010
Epoch 5/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 760ms/step - accuracy: 0.9431 - loss: 0.1366
Epoch 5: val_accuracy did not improve from 0.91827
326/326 ━━━━━━━━━━━━━━━━━━━━ 274s 840ms/step - accuracy: 0.9461 - loss: 0.1319 - val_accuracy: 0.9135 - val_loss: 0.2312 - learning_rate: 0.0010
Epoch 6/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 0s 753ms/step - accuracy: 0.9542 - loss: 0.1226
Epoch 6: val_accuracy did not improve from 0.91827

Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
326/326 ━━━━━━━━━━━━━━━━━━━━ 271s 832ms/step - accuracy: 0.9509 - loss

## Save Final Model

In [54]:
model.save("pneumonia_final_Dibyendu.h5")
print("Training complete. Model saved.")


Training complete. Model saved.
